TODO :
Implémentation
Factory pour wake word et STT
Dans les configs : pouvoir configurer le micro, les keywords, models wake word, model STT, le temps d'écoute/enregistrement

In [ ]:
# %sudo apt-get update
# %sudo apt-get install python3-pip python3-dev portaudio19-dev
# %pip install sounddevice vosk pvporcupine numpy


In [ ]:
# %wget https://alphacephei.com/vosk/models/vosk-model-small-fr-0.22.zip
# %unzip vosk-model-small-fr-0.22.zip


In [ ]:
# === Imports ===
import sounddevice as sd
import queue
import json
import numpy as np
import pvporcupine
from vosk import Model, KaldiRecognizer
import time

# === File audio (buffer) ===
audio_q = queue.Queue()

def audio_callback(indata, frames, time, status):
    """Callback pour remplir la file audio avec les données du micro."""
    if status:
        print(status)
    audio_q.put(indata.copy())

# === Initialisation Porcupine (wake word) ===
porcupine = pvporcupine.create(
    access_key="YTZtqOjD4eEEySY1rj2a3+cn2bNg38bJxzET+jn3SE5jpF9eiTx3hA==",
    model_path="wake_word_model/porcupine_params_de.pv",
    keyword_paths=["wake_words/glados_de_windows_v3_0_0.ppn"],
    # keywords=["jarvis"]
)  # Wake word "Jarvis"

# === Initialisation Vosk (STT) ===
model = Model("vosk-model-small-fr-0.22")
recognizer = KaldiRecognizer(model, 16000)

def detect_silence(audio, threshold=500):
    """Détecte le silence dans un tableau numpy d'audio int16.
    Retourne True si le niveau moyen est sous le seuil."""
    energy = np.abs(audio).mean()
    return energy < threshold

print("Assistant en écoute... (dites 'Jarvis' pour activer)")

# === Paramètres d'écoute ===
min_silence_len = 2  # Durée minimale de silence (en secondes) pour arrêter l'écoute

with sd.InputStream(
    channels=1,
    samplerate=16000,
    callback=audio_callback,
    blocksize=512
):
    while True:
        # Récupère un buffer audio du micro
        audio = audio_q.get()
        pcm = (audio.flatten() * 32767).astype(np.int16)

        # Découpe le buffer en frames pour Porcupine
        for i in range(0, len(pcm), porcupine.frame_length):
            frame = pcm[i:i+porcupine.frame_length]
            if len(frame) == porcupine.frame_length:
                keyword_index = porcupine.process(frame)
                if keyword_index >= 0:
                    print("Wake word détecté ! Parlez maintenant...")

                    # Vide la file audio pour éviter la redétection immédiate
                    while not audio_q.empty():
                        try:
                            audio_q.get_nowait()
                        except Exception:
                            break

                    frames = []
                    start_time = time.time()
                    max_duration = 8  # Durée max d'écoute (secondes)
                    silence_count = 0
                    silence_frames = int((min_silence_len * 16000) // 512)

                    # Écoute jusqu'à la fin de phrase (silence ou durée max)
                    while True:
                        audio = audio_q.get()
                        frames.append(audio)
                        pcm = (audio.flatten() * 32767).astype(np.int16)
                        recognizer.AcceptWaveform(pcm.tobytes())

                        if detect_silence(pcm):
                            silence_count += 1
                        else:
                            silence_count = 0

                        if silence_count >= silence_frames or (time.time() - start_time > max_duration):
                            break

                    # Résultat de la reconnaissance vocale
                    result = json.loads(recognizer.Result())
                    print("Texte:", result.get("text", ""))

                    break  # Sort du for, donc du buffer courant, et reprend la boucle principale

        else:
            continue  # Si aucun wake word détecté, on passe au buffer suivant
        # Si on a break le for (wake word détecté), on ne traite pas le reste du buffer et on reprend la boucle principale

Assistant en écoute... (dites 'Jarvis' pour activer)
Wake word détecté ! Parlez maintenant...
Texte: stop
Arrêt de l'assistant.
Wake word détecté ! Parlez maintenant...
Texte: stop
Arrêt de l'assistant.
Wake word détecté ! Parlez maintenant...
Texte: shutdown
Wake word détecté ! Parlez maintenant...
Texte: et speak english
Wake word détecté ! Parlez maintenant...
Texte: canaille speak english


KeyboardInterrupt: 

: 